In [1]:
from transformers import AutoTokenizer

model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

e:\PythonProjects\Summarization\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
e:\PythonProjects\Summarization\.venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice t

In [2]:
inputs = tokenizer("I loved reading the Hunger Games!")
inputs

{'input_ids': [336, 259, 28387, 11807, 287, 62893, 295, 12507, 309, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [3]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['▁I', '▁', 'loved', '▁reading', '▁the', '▁Hung', 'er', '▁Games', '!', '</s>']

In [4]:
max_input_length = 512
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["text"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["title"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [5]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# Load the dataset
news_train = load_dataset('csv', data_files='../data/train.csv')
news_test = load_dataset('csv', data_files='../data/test.csv')
news_validate = load_dataset('csv', data_files='../data/validate.csv')

# Split the dataset into train and test sets


In [6]:
def select_columns(example):
    return {'title': example['title'], 'text': example['text']}

selected = news_train.map(select_columns, remove_columns=['Unnamed: 0', 'url', 'topic', 'tags', 'date'])
selected_test = news_test.map(select_columns, remove_columns=['Unnamed: 0', 'url', 'topic', 'tags', 'date'])
selected_validate = news_validate.map(select_columns, remove_columns=['Unnamed: 0', 'url', 'topic', 'tags', 'date'])

In [7]:
tokenized_train = selected.map(preprocess_function, batched=True)
tokenized_test = selected_test.map(preprocess_function, batched=True)
tokenized_validate = selected_validate.map(preprocess_function, batched=True)

Map: 100%|██████████| 2000/2000 [00:01<00:00, 1963.01 examples/s]


In [8]:
tokenized_validate['train']

Dataset({
    features: ['title', 'text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2000
})

In [56]:
from datasets import DatasetDict

tokenized = DatasetDict()


tokenized['train'] = tokenized_train['train']
tokenized['test'] = tokenized_test['train']
tokenized['validate'] = tokenized_validate['train']

Метрика
ROUGE

Recall = Number of over lapping words
 / Total number of words in reference summary

​


Precision = 
Number of over lapping words /
Total number of words in generated summary
​
 



In [11]:
import evaluate

rouge_score = evaluate.load("rouge")

In [12]:
generated_summary = "I absolutely loved reading the Hunger Games"
reference_summary = "I loved reading the Hunger Games"

In [13]:
scores = rouge_score.compute(
    predictions=[generated_summary], references=[reference_summary]
)
scores

{'rouge1': 0.923076923076923,
 'rouge2': 0.7272727272727272,
 'rougeL': 0.923076923076923,
 'rougeLsum': 0.923076923076923}

In [14]:
from nltk.tokenize import sent_tokenize
import nltk

nltk.download("punkt")

def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])


print(three_sentence_summary(tokenized['train']['text'][0]))

Американский бомбардировщик-невидимка F-117 "Nighthawk" вызвал неподдельный интерес посетителей авиасалона ILA-2000, открывшегося во вторник в Берлинском аэропорту Schoenefeld.
Русские тоже представили на салоне военные МиГ-29, совершив на них беспосадочный перелет со своих аэродромов, отмечает РИА "Новости".
Среди участников берлинского авиасалона 940 фирм из 38 стран мира, всего на нем представлено более трехсот новейших летательных аппаратов.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tigra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(text) for text in dataset["text"]]
    return metric.compute(predictions=summaries, references=dataset["text"])

In [16]:
import pandas as pd

score = evaluate_baseline(tokenized['train'], rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(score[rn] * 100, 2)) for rn in rouge_names)
rouge_dict

{'rouge1': 50.65, 'rouge2': 33.93, 'rougeL': 50.65, 'rougeLsum': 49.61}

Дообучение mT5 с API Trainer

In [17]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [57]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 8
# Выводим потери при обучении по каждой эпохе
logging_steps = len(tokenized['train']) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-amazon-en-es",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=True,
)

In [19]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Декодируем сгенерированные резюме в текст
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Заменяем -100 в метках, поскольку мы не можем их декодировать
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Декодируем эталонные резюме в текст
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE ожидает символ новой строки после каждого предложения
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Вычисляем оценки ROUGE
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Извлекаем медианные оценки
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [20]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [58]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validate"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

e:\PythonProjects\Summarization\.venv\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [59]:
trainer.train()
trainer.evaluate()

  0%|          | 2/6000 [00:17<14:14:58,  8.55s/it]

KeyboardInterrupt: 

Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии. 


Result: На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии.